In [5]:
import os
import nltk
import string
from sklearn.externals import joblib
import pandas as pd
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors

scopes = ["https://www.googleapis.com/auth/youtube.force-ssl"]


def Get_Next_Comment(video_Id,api_service_name,api_version,DEVELOPER_KEY,nextPageToken):
    

    youtube = googleapiclient.discovery.build(
        api_service_name, api_version, developerKey = DEVELOPER_KEY)
    request = youtube.commentThreads().list(
        part="snippet",
        videoId=video_Id,
        maxResults="100",
        pageToken = nextPageToken
    )
    response = request.execute()
    response['nextPageToken']=response.get('nextPageToken',0)
    if response['nextPageToken'] !=0:
        for i in response['items']:
            comment.append(i['snippet']['topLevelComment']['snippet']['textDisplay'])
            comment_id.append(i['id'])
        Get_Next_Comment(video_Id,api_service_name,api_version,DEVELOPER_KEY,response['nextPageToken'])
        
    
    else:
        for i in response['items']:
            comment.append(i['snippet']['topLevelComment']['snippet']['textDisplay'])
            comment_id.append(i['id'])
        return comment
    
def Request_Comment(youtube):
   
    request = youtube.commentThreads().list(
        part="snippet",
        videoId=video_Id,
        maxResults="100"
    )
    response = request.execute()
    response['nextPageToken']=response.get('nextPageToken',0)
    
    
    for i in response['items']:
        comment_id.append(i['id'])
        comment.append(i['snippet']['topLevelComment']['snippet']['textDisplay'])
    
    if response['nextPageToken'] !=0:
      
        Get_Next_Comment(video_Id,api_service_name,api_version,DEVELOPER_KEY,response['nextPageToken'])
    
def Comment_Cleaning():
    punctuation_edit = string.punctuation.replace('\'','') +"0123456789"
    outtab = "                                         "
    trantab = str.maketrans(punctuation_edit, outtab)

    from stop_words import get_stop_words
    stop_words = get_stop_words('english')
    stop_words.append('')

    for x in range(ord('b'), ord('z')+1):
        stop_words.append(chr(x))


    from nltk.stem import PorterStemmer, WordNetLemmatizer

    lemmatiser = WordNetLemmatizer()
    stemmer = PorterStemmer()
    nltk.download('wordnet')
    
    for i in range(len(comment)):
        comment[i] = comment[i].lower().translate(trantab)
        l = []
        for word in comment[i].split():
            l.append(stemmer.stem(lemmatiser.lemmatize(word,pos="v")))
        comment[i] = " ".join(l)
    

    

def Comment_Classify():
    count_vector = joblib.load("Joblib/vector.joblib")
    clf = joblib.load("Joblib/model.joblib")
    tf_test = count_vector.transform(comment)
    predict =clf.predict(tf_test)
    dataset = pd.DataFrame({'comment_id':comment_id,'comment':comment, 'label':predict})
    print(dataset[dataset['label'] == 1])
    toxic_list=dataset[dataset['label'] == 1]['comment_id'].tolist()
    toxic_id=""
    for i in toxic_list:
        if i != toxic_list[-1]:
            toxic_id = toxic_id+i+','
        else:
            toxic_id = toxic_id+i
        
    return toxic_id
    

def Comment_Delete(youtube,toxic_id):

    request2 = youtube.comments().setModerationStatus(
        id=toxic_id,
        moderationStatus="rejected",
        banAuthor=False
    )
    request2.execute()


def main():
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

    api_service_name = "youtube"
    api_version = "v3"
    
    # Please put the path of your own credentail file here
    client_secrets_file = "client_secret.json"
    

    # Get credentials and create an API client
    flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
        client_secrets_file, scopes)
    credentials = flow.run_console()
    youtube = googleapiclient.discovery.build(
        api_service_name, api_version, credentials=credentials)
    
    Request_Comment(youtube)
    Comment_Cleaning()
    toxic_id =Comment_Classify()
    if toxic_id!="":
        Comment_Delete(youtube,toxic_id)
    else:
        print("There is no toxic comment from this video!")
    
    
    
    
if __name__ == "__main__":
    
    #Please put the video ID here
    video_Id=""  
    
    comment =[]
    comment_id=[]    
    main()

    

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=1040892163931-ehpeatruudvlqi9qtgii1pk8g29vj823.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=Pj7q0UxgyZdXk01fpbuJelkwEmVFlB&prompt=consent&access_type=offline
Enter the authorization code: 4/uAGybHUKlQ8HKQrRED3Z98rpCBUJlOxVLpKflBKTmX48E30LxtdavZg


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\cdrgv\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


                   comment_id                                   comment  label
0  UgxIW33Vz2DpB9fKOQB4AaABAg                 fukc you guy like a sheet      1
1  UgzyJ7HGJ2aIIX2CCC14AaABAg                 the quliti be like a crap      1
2  UgymQDqlLOgkZ3-mhwZ4AaABAg  the qualiti of your video be like a shit      1
3  UgyxT5PX5-jyl0D4UwB4AaABAg                                     idiot      1
